In [ ]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

In [ ]:
# Generate summary statistics
df_market_data.describe()

In [ ]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

In [ ]:
df_market_data.info()


---

### Prepare the Data

In [ ]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
df_market_data_scaled = StandardScaler().fit_transform(
    df_market_data[["price_change_percentage_24h","price_change_percentage_7d","price_change_percentage_14d",
                    "price_change_percentage_30d","price_change_percentage_60d",
                    "price_change_percentage_200d","price_change_percentage_1y"]]
)

# Display the first three rows of scaled data
df_market_data_scaled[0:3]

In [ ]:
# Create a DataFrame with the scaled data
df_market_data_scaled = pd.DataFrame(
    df_market_data_scaled,
    columns=["price_change_percentage_24h","price_change_percentage_7d","price_change_percentage_14d",
            "price_change_percentage_30d","price_change_percentage_60d",
            "price_change_percentage_200d","price_change_percentage_1y"],
)

# Copy the crypto names from the original data
df_market_data_scaled["coinid"] = df_market_data.index
# Set the coinid column as index
df_market_data_scaled = df_market_data_scaled.set_index("coinid")

# Display sample data
df_market_data_scaled.head()

---

### Find the Best Value for k Using the Original Data.

In [ ]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1, 11))

In [ ]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list
for i in k:
    k_model = KMeans(n_clusters=i, random_state=0)
    k_model.fit(df_market_data_scaled)
    inertia.append(k_model.inertia_)
    

In [ ]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)

# Create a DataFrame with the data to plot the Elbow curve
df_elbow.head()

In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
df_elbow.hvplot.line(
    x="k",
    y="inertia",
    title="Elbow Curve",
    xticks=k
)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** 4

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [ ]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4, random_state=0)

In [ ]:
# Fit the K-Means model using the scaled data
model.fit(df_market_data_scaled)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the scaled data
market_clusters = model.predict(df_market_data_scaled)   

# Print the resulting array of cluster values.
print(market_clusters)

In [ ]:
# Create a copy of the original DataFrame
df_market_data_scaled_copy = df_market_data_scaled.copy()

In [ ]:
# Add a new column to the DataFrame with the predicted clusters
df_market_data_scaled_copy["predicted_clusters"] = market_clusters

# Display sample data
df_market_data_scaled_copy.head()

In [ ]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
df_market_data_scaled_copy.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by="predicted_clusters",
    hover_cols=["coin_id"]
)

---

### Optimize Clusters with Principal Component Analysis.

In [ ]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)

In [ ]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
market_pca_data = pca.fit_transform(df_market_data_scaled)
# View the first five rows of the DataFrame. 
pca_df = pd.DataFrame(market_pca_data, columns=["PC1", "PC2", "PC3"])
pca_df[:5]

In [ ]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
explained_variance = pca.explained_variance_ratio_
print(f"{round(sum(explained_variance) * 100, 2)}% of the information is preserved by the principal components.")

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer**
To find the total explained variance of the three principal components,
we can sum the explained variance ratio of each principal component then multiply by 100 to showcase the percentage. 
Using the function **print(f"{round(sum(explained_variance) * 100, 2)}%")** we 
find that 89.5% of the information is preserved by the principal components.
            

In [ ]:
# Create a new DataFrame with the PCA data.
df_market_pca = pd.DataFrame(
    market_pca_data, columns=["PC1", "PC2", "PC3"]
)
# Copy the crypto names from the original data
df_market_pca["coinid"] = df_market_data.index

# Set the coinid column as index
df_market_pca = df_market_pca.set_index("coinid")

# Display sample data
df_market_pca.head()

---

### Find the Best Value for k Using the PCA Data

In [ ]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1, 11))

In [ ]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list
for i in k:
    k_model = KMeans(n_clusters=i,random_state=0)
    k_model.fit(df_market_pca)
    inertia.append(k_model.inertia_)

In [ ]:
# Create a dictionary with the data to plot the Elbow curve
market_elbow_data = {"k":k, "inertia": inertia}
# Create a DataFrame with the data to plot the Elbow curve
df_market_pca_elbow = pd.DataFrame(market_elbow_data)
df_market_pca_elbow.head()  

In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
df_market_pca_elbow.hvplot.line(
    x="k",
    y="inertia",
    title="Elbow Curve",
    xticks=k
)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** 4


* **Question:** Does it differ from the best k value found using the original data? 

  * **Answer:** No the k value does not differ from the original data.

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [ ]:
# Initialize the K-Means model using the best value for k
pca_model = KMeans(n_clusters=4,random_state=0)

In [ ]:
# Fit the K-Means model using the PCA data
pca_model.fit(df_market_pca)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the PCA data
pca_market_clusters = pca_model.predict(df_market_pca)
# Print the resulting array of cluster values.
print(pca_market_clusters)

In [ ]:
# Create a copy of the DataFrame with the PCA data
market_cluster_pca_copy = df_market_pca.copy()

# Add a new column to the DataFrame with the predicted clusters
market_cluster_pca_copy["pc_predicted_clusters"] = pca_market_clusters  
# Display sample data
market_cluster_pca_copy.head()

In [ ]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
market_cluster_pca_copy.hvplot.scatter(
    x="PC1",
    y="PC2",
    by="pc_predicted_clusters",
    hover_cols=["coin_id"]
)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [ ]:
# Composite plot to contrast the Elbow curves
df_elbow.hvplot.line(
    x="k",
    y="inertia",
    title="Elbow Curve (Market Data)",
    xticks=k
) * df_market_pca_elbow.hvplot.line(
    x="k",
    y="inertia",
    title="Elbow Curve (PCA Data)",
    xticks=k
)

In [ ]:
# Composite plot to contrast the clusters
# YOUR CODE HERE!
df_market_data_scaled_copy.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by="predicted_clusters",
    hover_cols=["coin_id"]
) + market_cluster_pca_copy.hvplot.scatter(
    x="PC1",
    y="PC2",
    by="pc_predicted_clusters",
    hover_cols=["coin_id"]    
)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:**    With PCA, we can reduce the number of features to three principal components. This reduction in features allows us to visualize the data in a three-dimensional space. The PCA analysis 
    shows that the information is preserved by the principal components. Using PCA we can still explain 89.5% of the information with three principal components. 
    
    The Elbow curve for the PCA data shows that the optimal value for k is 4. The PCA analysis allows us to reduce the number of features and still obtain a good clustering result.